# Scraping 3D Juegos

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from newspaper import Source
from newspaper import fulltext

## Scraping Articles Site 

In [188]:
review_url = 'https://www.3djuegos.com/novedades/analisis/juegos-ps4/0f37f0f0/fecha/'
html = requests.get(review_url).content
html[:50]

b'<!DOCTYPE html><html lang="es"><head>\n<!--[if lt I'

In [190]:
soup = BeautifulSoup(html, 'lxml')
articles = soup.find_all('h2')

In [24]:
#Extraer títulos

titles = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        titles.append(a.find('a')['title'])
    
titles[0:5]

['FIFA 21 - Análisis',
 'Genshin Impact - Análisis',
 "Crash Bandicoot 4: It's About Time - Análisis",
 'Star Wars: Squadrons - Análisis',
 'Kirby Fighters 2 - Análisis']

In [16]:
#Extraer links

links = [a.find('a')['href'] for a in articles]
    
links[0:4]

['https://www.3djuegos.com/juegos/analisis/37068/0/fifa-21/',
 'https://www.3djuegos.com/juegos/analisis/36571/0/genshin-impact/',
 'https://www.3djuegos.com/juegos/analisis/37086/0/crash-bandicoot-4-its-about-time/',
 'https://www.3djuegos.com/juegos/analisis/36317/0/star-wars-squadrons/']

## Page parsing and link retrieving function for 3D Juegos

In [3]:
def tdjuegos_link_retrieve(num_pages):
    links = []
    titles = []
    
    # pages parser
    for i in range(num_pages):
        url = f"https://www.3djuegos.com/novedades/analisis/juegos/{i+100}pf0f0f0/fecha/"

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find_all('h2')

        # links retrieve
        for a in articles:
            links.append(a.find('a')['href']) 
            titles.append(a.find('a')['title'])
    
    #Deleting non review links
#    for link in links:
#        if 'analisis' not in link:
#            links.remove(link)
                
    return links, titles


In [4]:
num_pages = 50

links, titles = tdjuegos_link_retrieve(num_pages)

In [11]:
len(links)

1498

## Scraping Single Review

In [138]:
review_url = 'https://www.3djuegos.com/juegos/analisis/37068/0/fifa-21/'
review_html = requests.get(review_url).content
review_html[:50]

b"<!DOCTYPE html>\n<html lang='es'>\n<head>\n<meta prop"

In [139]:
soup = BeautifulSoup(review_html, 'lxml')

In [140]:
p_tags = soup.find('div', {'class': 'lh27 url_lineas article_body0 mar_temp_0'}).find_all('p')

In [141]:
review = [tag.text for tag in p_tags]
review = ' '.join(review)

In [143]:
review[0:333]

'FIFA 21 no será la entrega más recordada de la saga de simulación futbolística de EA Sports. Ligeras modificaciones jugables, técnicas y modalidades subrayan un pensamiento generalizado de toda la comunidad: hay que dar un buen repaso a la franquicia en la nueva generación... y quitar a Manolo Lama y Paco González en la versión esp'

In [144]:
author = soup.find('a', {'class': 'c7 n'}).text
author

'Toni Piedrabuena'

In [82]:
# Author
'''def tdj_author():
    author = 'None'
    
    try:
        author = soup.find('p', {'class': 'art-aut-wr'}).find('a').text.strip('\n')

    except AttributeError:
        pass
    
    try:
        author = soup.find('li', {'class': 'art-aut-wr'}).text.strip('\n')

    except AttributeError:
        pass

    return author'''

In [9]:
# No Genre, No Company and Platform

def tdjuegos_platform(soup):
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c1o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
   
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c2o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    
    except AttributeError:
        pass

    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c3o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c4o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c5o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
 
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c6o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass
    
    try:
        platform_soup = soup.find('div', {'class': 'he30 s13 c34o cb mar_t3 mar_l2 d_fl a_n a_cFh fftit'}).find_all('span')
    
    except AttributeError:
        pass

    finally:
        
        platform = [plat.contents[0] for plat in platform_soup[2:]]        
        platforms = ' '.join(platform)
            
    return platforms

In [174]:
tdjuegos_platform(soup)

'PC PS4 PS5 XSeries XOne Switch'

In [116]:
#Score

score = soup.find('div', {'class': 'nota_ana_3 fftext b nota_interior2'}).text
score = score.replace(',','.')
score = float(score)

6.0

## Create columns

In [10]:

def tdjuegos_dict(links, titles):
    reviews_dict = {}
    i = 0

    for link, title in zip(links, titles):
        try:
            
            #Request content and wait
            review_html = requests.get(link).content

            #Avoid get banned and timeout
            time.sleep(1)

            #Make a Soup and a Newspacer build
            soup = BeautifulSoup(review_html, 'lxml')

            #Author - From Scraping
            author = soup.find('a', {'class': 'c7 n'}).text

            #Company, Genre & Platform - From Scraping
            genre = 'None'
            company = 'None'
            platform = 'None'

            platform = tdjuegos_platform(soup)

            #Text & Cleaning - From Scraping
            p_tags = p_tags = soup.find('div', {'class': 'lh27 url_lineas article_body0 mar_temp_0'}).find_all('p')
            review = [tag.text for tag in p_tags]
            review = ' '.join(review)

            #Score & Clean & Transform - From Scraping
            score = 7

            try:
                score = soup.find('div', {'class': 'nota_ana_3 fftext b nota_interior2'}).text
                score = score.replace(',','.')
                
            except AttributeError:
                pass
            
            score = float(score)
            
            #Add to a dict
            reviews_dict[i] = {'site': '3D Juegos',
                               'url_link': link,
                               'author': author,
                               'game': title,
                               'company': company,
                               'genre': genre,
                               'platform': platform,
                               'text': review,
                               'score': score}
        
        except AttributeError:
            print('error', i)
            pass
        
        i = i+1
        if i % 25 == 0:
            print(i, ': ', link)

    return reviews_dict

In [13]:
result_tdjuegos = tdjuegos_dict(links[28:], titles[28:])

25 :  https://www.3djuegos.com/juegos/analisis/11986/0/blocks-that-matter/
50 :  https://www.3djuegos.com/juegos/analisis/9729/0/capitan-america-super-soldier/
75 :  https://www.3djuegos.com/juegos/analisis/7974/0/fear-3/
100 :  https://www.3djuegos.com/juegos/analisis/1637/0/duke-nukem-forever/
125 :  https://www.3djuegos.com/juegos/analisis/11166/0/hydrophobia-prophecy/
150 :  https://www.3djuegos.com/juegos/analisis/11100/0/bulletstorm-gun-sonata/
175 :  https://www.3djuegos.com/juegos/analisis/10131/0/ghostbusters-sanctum-of-slime/
200 :  https://www.3djuegos.com/juegos/analisis/5829/0/homefront/
225 :  https://www.3djuegos.com/juegos/analisis/9450/0/torchlight/
250 :  https://www.3djuegos.com/juegos/analisis/9012/0/kirby39s-epic-yarn/
275 :  https://www.3djuegos.com/juegos/analisis/4559/0/dead-space-2/
300 :  https://www.3djuegos.com/juegos/analisis/7426/0/tron-evolution/
325 :  https://www.3djuegos.com/juegos/analisis/9867/0/pac-man-champion39s-edition-dx/
350 :  https://www.3dju

### Create DataFrame

In [14]:
tdjuegos = pd.DataFrame.from_dict(result_tdjuegos, orient='index')

tdjuegos

,site,url_link,author,game,company,genre,platform,text,score
0,3D Juegos,https://www.3djuegos.com/juegos/analisis/11738...,Alberto Pastor,Hard Reset - Análisis,None,None,PC,Prometieron un juego de acción de la vieja esc...,7.9
1,3D Juegos,https://www.3djuegos.com/juegos/analisis/9594/...,Jesús Bella,Rise of Nightmares - Análisis,None,None,X360,Terror y Kinect. Dos palabras con significados...,5.6
2,3D Juegos,https://www.3djuegos.com/juegos/analisis/9086/...,Alberto Pastor,Rock of Ages - Análisis,None,None,PC PS3 X360,¿Qué tienen en común personajes históricos com...,8.0
3,3D Juegos,https://www.3djuegos.com/juegos/analisis/9815/...,Alberto Pastor,Rock of Ages - Análisis,None,None,PC PS3 X360,¿Qué tienen en común personajes históricos com...,8.0
4,3D Juegos,https://www.3djuegos.com/juegos/analisis/9816/...,Alberto Pastor,Rock of Ages - Análisis,None,None,PC PS3 X360,¿Qué tienen en común personajes históricos com...,8.0
...,...,...,...,...,...,...,...,...,...
1465,3D Juegos,https://www.3djuegos.com/juegos/analisis/2903/...,Jesús Bella,Viva Piñata: Pocket Paradise - Análisis,None,None,PC X360 DS,Las coloridas piñatas de papel “made in Rare” ...,7.8
1466,3D Juegos,https://www.3djuegos.com/juegos/analisis/4374/...,David Hernández,Pool Party - Análisis,None,None,Wii,Los billares siempre han sido una gran escapat...,5.2
1467,3D Juegos,https://www.3djuegos.com/juegos/analisis/2182/...,Jesús Bella,Spore Creatures - Análisis,None,None,PC DS Mac iOS,Will Wright redimensiona las teorías de la evo...,7.0
1468,3D Juegos,https://www.3djuegos.com/juegos/analisis/3945/...,Jesús Bella,Los Sims 2: Vecinos y mascotas - Análisis,None,None,DS,Los Sims se acercan cada vez más a su tercer g...,6.0


In [18]:
#tdjuegos.to_csv('../data/tdjuegos_3000_4470l.csv', index=False)

In [19]:
tdjuegos.iloc[::-1].head(50)

,site,url_link,author,game,company,genre,platform,text,score
1469,3D Juegos,https://www.3djuegos.com/juegos/analisis/1886/...,Jesús Bella,Infinite Undiscovery - Análisis,None,None,X360,Con el objetivo de descubrir el infinito en el...,6.9
1468,3D Juegos,https://www.3djuegos.com/juegos/analisis/3945/...,Jesús Bella,Los Sims 2: Vecinos y mascotas - Análisis,None,None,DS,Los Sims se acercan cada vez más a su tercer g...,6.0
1467,3D Juegos,https://www.3djuegos.com/juegos/analisis/2182/...,Jesús Bella,Spore Creatures - Análisis,None,None,PC DS Mac iOS,Will Wright redimensiona las teorías de la evo...,7.0
1466,3D Juegos,https://www.3djuegos.com/juegos/analisis/4374/...,David Hernández,Pool Party - Análisis,None,None,Wii,Los billares siempre han sido una gran escapat...,5.2
1465,3D Juegos,https://www.3djuegos.com/juegos/analisis/2903/...,Jesús Bella,Viva Piñata: Pocket Paradise - Análisis,None,None,PC X360 DS,Las coloridas piñatas de papel “made in Rare” ...,7.8
1464,3D Juegos,https://www.3djuegos.com/juegos/analisis/3029/...,Álvaro Castellano,Smash Court Tennis 3 - Análisis,None,None,X360 PSP,"Smash Court Tennis va por la tercera entrega, ...",5.1
1463,3D Juegos,https://www.3djuegos.com/juegos/analisis/3047/...,Jesús Bella,Dragon Quest IV: Capítulos de los Elegidos - A...,None,None,DS Android iOS PS1 NES,"El tiempo pasa, los avances no cesan, los vide...",8.8
1462,3D Juegos,https://www.3djuegos.com/juegos/analisis/1863/...,Álvaro Castellano,Star Wars: El Poder de la Fuerza - Análisis,None,None,PS3 X360 Wii PSP DS PS2,Star Wars debuta en la Next-Gen y lo hace con ...,7.9
1461,3D Juegos,https://www.3djuegos.com/juegos/analisis/1864/...,Álvaro Castellano,Star Wars: El Poder de la Fuerza - Análisis,None,None,PS3 X360 Wii PSP DS PS2,Star Wars debuta en la Next-Gen y lo hace con ...,7.9
1460,3D Juegos,https://www.3djuegos.com/juegos/analisis/4224/...,David Hernández,Futbolín - Análisis,None,None,Wii,Parecía imposible ver un juego basado en el fu...,6.0


In [17]:
len(tdjuegos)

1470